In [1]:
import os
import sys

sys.path.append("../../common/src")
sys.path.append("../src")
os.chdir("../src")

In [2]:
from langchain.agents import Tool, AgentExecutor, BaseMultiActionAgent, AgentOutputParser
from langchain.agents.chat.base import ChatAgent
from langchain.prompts import StringPromptTemplate
from langchain.llms import OpenAI, VertexAI
from langchain.chat_models import ChatVertexAI, ChatOpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.chains import LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, OutputParserException
import re

In [3]:
!export PROJECT_ID="gcp-mira-demo"
project = "gcp-mira-demo"
os.environ["PROJECT_ID"] = project
PROJECT_ID = project

In [4]:
import config
from config import (VERTEX_LLM_TYPE_BISON_CHAT,
                    VERTEX_LLM_TYPE_BISON_TEXT,
                    OPENAI_LLM_TYPE_GPT3_5,
                    OPENAI_LLM_TYPE_GPT4,
                    COHERE_LLM_TYPE,
                    LLM_BACKEND_ROBOT_USERNAME,
                    LLM_BACKEND_ROBOT_PASSWORD)
from common.utils.token_handler import UserCredentials

config.RULES_ENGINE_BASE_URL = "https://gcp-mira-demo.cloudpssolutions.com/rules-engine/api/v1"
config.auth_client = UserCredentials(LLM_BACKEND_ROBOT_USERNAME,
                              LLM_BACKEND_ROBOT_PASSWORD,
                              "https://gcp-mira-demo.cloudpssolutions.com")

INFO: [src/config.py:49 - <module>()] Namespace File not found, setting job namespace as default
INFO: [src/config.py:81 - get_environ_flag()] ENABLE_GOOGLE_LLM = True
INFO: [src/config.py:81 - get_environ_flag()] ENABLE_OPENAI_LLM = True
INFO: [src/config.py:81 - get_environ_flag()] ENABLE_COHERE_LLM = True
INFO: [src/config.py:178 - <module>()] LLM types loaded ['OpenAI-GPT3.5', 'OpenAI-GPT4', 'Cohere', 'VertexAI-Text', 'VertexAI-Chat', 'VertexAI-Chat']


In [5]:
from common.models import Agent
from common.models.agent import AgentType, UserPlan, PlanStep
from services.agents.agent_service import get_agent_config, get_plan_agent_config, get_all_agents

In [6]:
from services.agents.agents import TaskAgent, PlanAgent
#casey_plan_agent = TaskAgent(VERTEX_LLM_TYPE_BISON_TEXT)
#casey_plan_agent = TaskAgent(VERTEX_LLM_TYPE_BISON_CHAT)
#casey_plan_agent = TaskPlanAgent(OPENAI_LLM_TYPE_GPT3_5)
casey_plan_agent = TaskAgent(OPENAI_LLM_TYPE_GPT4)
#casey_plan_agent = TaskPlanAgent(COHERE_LLM_TYPE)

In [7]:
prompt = """Compose and send an email to all the medicaid applicants that are missing income verification asking them to provide a pay stub from their employers"""

In [8]:
user_id = "5nJrkPWa3D0yCKA853mD"

In [9]:
#agent_executor.run(agent_inputs)

In [10]:
from services.agents.agent_service import agent_plan, parse_plan

In [11]:
plan = agent_plan("Plan", prompt, user_id)

user_plan = plan[1]
plan_steps=user_plan.plan_steps
chat_history = []

print("\n\nPlan Steps: ")
for step in plan_steps:
    description = PlanStep.find_by_id(step).description
    print (description)
    chat_history.append(description)

INFO: [agents/agent_service.py:156 - agent_plan()] Starting with plan for agent_name=[Plan], prompt=[Compose and send an email to all the medicaid applicants that are missing income verification asking them to provide a pay stub from their employers], user_id=[5nJrkPWa3D0yCKA853mD], chat_history=[None]
INFO: [agents/agent_service.py:114 - run_agent()] Running Plan agent with prompt=[Compose and send an email to all the medicaid applicants that are missing income verification asking them to provide a pay stub from their employers] and chat_history=[None]
INFO: [agents/agent_service.py:123 - run_agent()] Available tools=[gmail_tool, docs_tool, calendar_tool, search_tool, query_tool]
INFO: [agents/agents.py:100 - load_agent()] Successfully loaded PlanAgent agent.
INFO: [agents/agent_service.py:135 - run_agent()] Running agent executor.... 
INFO: [agents/agent_service.py:137 - run_agent()] Agent Plan generated output=[Task: Compose and send an email to all the Medicaid applicants that are 

In [12]:
tools = casey_plan_agent.get_tools()
print(tools)

agent = casey_plan_agent.load_agent()

prompt = """Send an email to all the medicaid applicants that are missing income verification asking them to provide a pay stub from their employers"""

prompt = """Run the plan in the chat history provided below."""
agent_executor = AgentExecutor.from_agent_and_tools(
  agent=agent, 
    tools=tools,
    verbose=True)
agent_inputs = {
    "input": prompt,
    "chat_history": chat_history
}

result = agent_executor.run(agent_inputs)


[StructuredTool(name='gmail_tool', description='gmail_tool(email_text: str) -> str - Send an email using Gmail', args_schema=<class 'pydantic.main.gmail_toolSchemaSchema'>, func=<function gmail_tool at 0x15fe19e40>), StructuredTool(name='docs_tool', description='docs_tool(content: str) -> str - Compose or create a document using Google Docs', args_schema=<class 'pydantic.main.docs_toolSchemaSchema'>, func=<function docs_tool at 0x15fe19ee0>), StructuredTool(name='calendar_tool', description='calendar_tool(date: str) -> str - Create and update meetings using Google Calendar', args_schema=<class 'pydantic.main.calendar_toolSchemaSchema'>, func=<function calendar_tool at 0x15fe19f80>), StructuredTool(name='search_tool', description='search_tool(query: str) -> str - Perform an internet search.', args_schema=<class 'pydantic.main.search_toolSchemaSchema'>, func=<function search_tool at 0x15fe1a340>), StructuredTool(name='query_tool', description='query_tool(query: str) -> str - Perform a qu

In [13]:
print(result)

The plan has been executed successfully. An email has been sent to the identified Medicaid applicant requesting income verification.


In [14]:
# agent_executor.agent.llm_chain.prompt
